In [1]:
from cvxopt import matrix, solvers
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import time

# Source - https://stackoverflow.com/a
# Posted by sjm, modified by community. See post 'Timeline' for change history
# Retrieved 2026-01-03, License - CC BY-SA 4.0
solvers.options['show_progress'] = False  # didn't know that it pastes with a ref but it's cool

CLASSES = np.array(["banana", "carrot", "cucumber", "mandarin", "tomato"])

EPS  = 1e-6
SEED = 462
np.random.seed(SEED)

data_path = os.path.join("..", "data_workflow_notebooks", "data", "tabular")

# Soft-Margin SVM

$$
min_\alpha \frac{1}{2}\sum_{m=1}^N\sum_{n=1}^N\alpha_n\alpha_my_ny_mx_n^Tx_m - \sum_{n=1}^N\alpha_n
$$

$$
s.t.\ \sum_{n=1}^Ny_n\alpha_n = 0
$$

$$
\ \ \ \ \ \ 0 \leq \alpha_n \leq C
$$

$$
n=1,\dots,N
$$

# QP Solver Representation

```python3
solvers.qp(Q, p, G, h, A, b)
```

Where the problem is:
$$
min_x \frac{1}{2}x^TQx + p^Tx
$$

$$
s.t.\ Gx \leq h
$$

$$
\ \ \ \ \ \ Ax = b
$$

# What to do

We need to convert Soft-Margin SVM representation into QP solver one.

In [2]:
class Dataset:
    """
    This class is taken directly from our previous submission.
    """
    def __init__(self, train_path=None, test_path=None):
        self.train_path = train_path
        self.test_path = test_path
    
    def load_csv(self, path):
        data = pd.read_csv(path).to_numpy()
        X, Y_str, filenames = data[:, :-2], data[:, -2:-1], data[:, -1]  # separate data and target
        n_examples = len(Y_str)
        Y = np.zeros(n_examples)
        for i in range(n_examples):
            category = Y_str[i]
            if   category == CLASSES[0]:
                Y[i] = 0
            elif category == CLASSES[1]:
                Y[i] = 1
            elif category == CLASSES[2]:
                Y[i] = 2
            elif category == CLASSES[3]:
                Y[i] = 3
            else:
                Y[i] = 4
        
        return X.astype(float), Y.astype(float), filenames
    
    def get_data(self):
        X_train, Y_train, filenames_train = self.load_csv(self.train_path)
        X_test , Y_test , filenames_test  = self.load_csv(self.test_path)
        
        return (X_train, Y_train, filenames_train), (X_test, Y_test, filenames_test)

# Soft-Margin SVM Implementation

In [3]:
class SoftMarginSVM:
    def __init__(self, C=1):
        self.weights    = None
        self.bias       = None
        self.alpha_star = None
        self.sv_indices = None
        self.ol_indices = None  # outlier indices
        self.cls        = None  # this is used in the prediction phase to decide which class is the prediction
        self.C          = C     # soft margin svm gets closer to hard when C gets greater
        
    def train(self, X, Y):
        n_examples, n_features = X.shape

        # we need to calculate Q using y_n*y_m and x_n^Tx_m
        Y = Y.reshape(-1, 1)    # to prevent np to give a scalar make it Nx1
        Y_mul = np.dot(Y, Y.T)  # NxN
        X_mul = np.dot(X, X.T)  # NxN

        Q = Y_mul * X_mul
        p = (np.ones(n_examples) * -1).reshape(-1, 1)

        # G is supposed to be (2N)xN
        G_first_half  = np.eye(n_examples) * -1  # greater than or equal to 0
        G_second_half = np.eye(n_examples)       # less than or equal to C
        G = np.vstack([G_first_half, G_second_half])
        h_first_half  = np.zeros(n_examples).reshape(-1, 1)
        h_second_half = (np.ones(n_examples) * self.C).reshape(-1, 1)
        h = np.vstack([h_first_half, h_second_half])

        A = Y.reshape(1, -1)
        b = 0.0  # cvxopt expects double-precision

        sol = solvers.qp(matrix(Q), matrix(p), matrix(G), matrix(h), matrix(A), matrix(b))

        # next: use sol["x"] to extract optimal alphas and then calculate w and b
        # calculate w*
        alpha_star      = np.array(sol["x"]).reshape(-1, 1)  # Nx1
        self.alpha_star = alpha_star
        weighted_label  = Y * alpha_star  # results in Nx1
        w_star          = np.dot(X.T, weighted_label)  # results in dx1 where d is the n_featuresy
        self.weights    = w_star

        # calculate b*
        epsilon = EPS  # since optimizers do not work with total precision
        sv_mask = ((alpha_star > epsilon) & (alpha_star < (self.C - epsilon))).flatten()  # flatten is used to prevent IndexError
        # numpy needs this to apply the mask row-wise
        Y_s     = Y[sv_mask]
        X_s     = X[sv_mask]
        pred    = np.dot(X_s, w_star)
        bias    = Y_s - pred
        self.bias = np.mean(bias)

        # detect sv indices
        self.sv_indices = np.where(sv_mask)[0]

        # detect outlier indices
        self.ol_indices = np.where((alpha_star >= self.C - epsilon))[0]

    def decision_function(self, X):
        return (np.dot(X, self.weights) + self.bias)

    def predict(self, X):
        return np.sign(self.decision_function(X))
    
    def hinge_loss(self, y_true, y_pred):
        return np.mean(np.maximum(0, 1 - y_true * y_pred))

In [ ]:
class SoftMarginSVM_OVA:
    def __init__(self, C=1):
        self.C       = C
        self.models  = []
        self.classes = None
        self.history = None

    def train(self, X_train, Y_train):
        self.classes = np.unique(Y_train)
        self.history = {}

        for cls in self.classes:
            print(f"--- training for class {cls} ---")
            Y_train_bin = np.where(Y_train == cls, 1, -1).astype(float)  # ready for binary classification

            model = SoftMarginSVM(self.C)
            model.cls = cls
            
            n_examples, n_features = X_train.shape

            model.train(X_train, Y_train_bin)
            y_pred = model.decision_function(X_train)
            train_loss = model.hinge_loss(Y_train_bin, y_pred)
            self.models.append(model)
            self.history[cls] = {"train_loss": train_loss}

    def decision_functions_all(self, X):
        return np.column_stack([model.decision_function(X) for model in self.models])
    
    def predict(self, X):
        decisions = self.decision_functions_all(X)
        return [self.models[idx].cls for idx in np.argmax(decisions, axis=1)]
    
    def find_farthest_points(self, X, filenames):
        farthest = {"C": self.C}
        scores = self.decision_functions_all(X)

        for i, cls_label in enumerate(self.classes):
            class_scores = scores[:, i]  # get the scores of the ith class
            best_idx = np.argmax(class_scores)

            farthest[CLASSES[int(cls_label)]] = {"best_index": best_idx, "filename": filenames[best_idx], "score": class_scores[best_idx]}

        return farthest
    
    def get_svs(self, filenames):
        svs = {"C": self.C}
        
        for model in self.models:
            cls = CLASSES[int(model.cls)]
            sv_indices = model.sv_indices
            ol_indices = model.ol_indices

            svs[cls] = {
                "sv_filenames": filenames[sv_indices].tolist(),
                "ol_filenames": filenames[ol_indices].tolist()
                }
        
        return svs
    
    def get_svs_same_cls(self, filenames):
        """
        we need to return sv indices with the same class to calculate their
        distances, similarities, etc. additionally, for debugging purposes
        returning filenames might be useful
        """
        svs_same_cls = {"C": self.C}

        for model in self.models:
            cls = CLASSES[int(model.cls)]
            sv_indices = model.sv_indices
            # find the indices that corresponds to a cls element
            mask = np.char.find(filenames[sv_indices].astype(str), cls) != -1
            sv_same_cls_indices = sv_indices[mask]

            # actually we do not need to create another key "svs_same_cls". we can remove it for further refinements
            svs_same_cls[cls] = {
                "svs_same_cls": list(zip(sv_same_cls_indices.tolist(), filenames[sv_same_cls_indices].tolist()))
            }

        return svs_same_cls

# Train With Various C Values

In [ ]:
dataset = Dataset(
    train_path=os.path.join(data_path, "train_processed.csv"),
    test_path=os.path.join(data_path, "test_processed.csv")
)

(X_train, Y_train, filenames_train), (X_test, Y_test, filenames_test) = dataset.get_data()

# we can train with different C values to evaluate their relative performances
svms   = []
C_vals = [
    # 0.1, 
    # 1, 
    10
    ]

for c in C_vals:
    svm = SoftMarginSVM_OVA(C=c)

    print(f"============== start training with C = {c} ==============")
    start_time = time.time()
    svm.train(X_train, Y_train)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"\nTotal Training Time: {training_time:.2f} seconds")
    svms.append(svm)

============== start training with C = 10 ==============
--- training for class 0.0 ---
--- training for class 1.0 ---
--- training for class 2.0 ---
--- training for class 3.0 ---
--- training for class 4.0 ---

Total Training Time: 1506.14 seconds


In [ ]:
for s in svms:
    c = s.C
    print(f"============== prediction with C = {c} ==============")
    Y_pred = s.predict(X_test)
    Y_pred = np.array(Y_pred)

    accuracy = np.mean(Y_pred == Y_test)
    print(f"Final Test Accuracy: {accuracy * 100:.2f}%\n")

    # save
    results_df = pd.DataFrame({'True_Label': Y_test, 'Predicted_Label': Y_pred})
    results_df.to_csv(f"svm_predictions_output{c}.csv", index=False)

============== prediction with C = 10 ==============
Final Test Accuracy: 91.03%



# Support Vector - Outliers - High Confidence (Farthest)

In [78]:
for cls in CLASSES:
    print(f"=========== {cls.upper()} ===========")
    for s in svms:
        svs_report  = s.get_svs(filenames_train)
        fart_report = s.find_farthest_points(X_train, filenames_train)
        print(f"=========== C = {svs_report["C"]} ===========")
        print("=========== support vectors ===========")
        print("support vectors:".upper(), svs_report[cls]["sv_filenames"])
        print("outliers:".upper(),svs_report[cls]["ol_filenames"])
        print("=========== farthest points ===========")
        print(fart_report[cls])
        print()
        print("-" * 60)

=========== BANANA ===========
=========== C = 10 ===========
=========== support vectors ===========
SUPPORT VECTORS: ['data/image/train/mandarin/0894.png', 'data/image/train/banana/0565.png', 'data/image/train/banana/0677.png', 'data/image/train/mandarin/0223.png', 'data/image/train/banana/0104.png', 'data/image/train/cucumber/0298.png', 'data/image/train/cucumber/0999.png', 'data/image/train/cucumber/0913.png', 'data/image/train/mandarin/0001.png', 'data/image/train/mandarin/0273.png', 'data/image/train/banana/0331.png', 'data/image/train/banana/0586.png', 'data/image/train/banana/0316.png', 'data/image/train/banana/0177.png', 'data/image/train/banana/0686.png', 'data/image/train/banana/0326.png', 'data/image/train/banana/0743.png', 'data/image/train/mandarin/0709.png', 'data/image/train/carrot/0770.png', 'data/image/train/mandarin/0717.png', 'data/image/train/banana/0408.png', 'data/image/train/banana/0734.png', 'data/image/train/carrot/1006.png', 'data/image/train/cucumber/0141.pn

# Distance Calculation of Different Class SVs

In [31]:
# svs_same_cls[cls] = {
#     "svs_same_cls": list(zip(sv_same_cls_indices.tolist(), filenames[sv_same_cls_indices].tolist()))
# }

def get_cls_svs(X, sv_dict):
    """
    return:
        - sv_data_points: a dictionary in the form of `{<cls_name>: <np_array_of_sv_features>}`
        - com_svs: a dictionary in the form of `{<cls_name>: <center_of_mass_of_svs>}`
    """
    sv_data_points = {}
    com_svs        = {}
    for cls in CLASSES:
        cls_indices_list    = [tup[0] for tup in sv_dict[cls]["svs_same_cls"]]
        data_points         = X[cls_indices_list]
        sv_data_points[cls] = data_points
        com_svs[cls]        = np.mean(data_points, axis=0)

    return sv_data_points, com_svs

def calculate_sv_dist_pw(com_svs, classes):
    """
    calculate pairwise distances of sv center of masses
    """
    sv_dist_pw = {}
    for cls0 in classes:
        classes = np.delete(classes, np.where(classes == cls0))
        for cls1 in classes:
            sv_dist_pw[f"{cls0}-{cls1}"] = np.linalg.norm(com_svs[cls0] - com_svs[cls1])

    return sv_dist_pw

In [63]:
c_10_svm = svms[0]  # there are 5 svms in c_10_svm.models
sv_dict = c_10_svm.get_svs_same_cls(filenames_train)

sv_data_points, com_svs = get_cls_svs(X_train, sv_dict)

# print(sv_data_points)
# print(com_svs["banana"])

sv_dist_pw = calculate_sv_dist_pw(com_svs, CLASSES)
sv_dist_pw = dict(sorted(sv_dist_pw.items(), key=lambda pair: pair[1]))  # sort the dictionart in distance ascending order

print("="*10, "DISTANCES", "="*10)

for k, v in sv_dist_pw.items():
    print(f"{k:<{18}} | {v:.4f}")

print("="*31)

========== DISTANCES ==========
carrot-cucumber    | 1.6727
mandarin-tomato    | 1.8552
carrot-mandarin    | 2.3193
banana-carrot      | 2.4131
cucumber-mandarin  | 2.7752
banana-cucumber    | 2.9387
carrot-tomato      | 3.2212
cucumber-tomato    | 3.4213
banana-mandarin    | 3.6384
banana-tomato      | 4.6404


# Confusion

In [64]:
# now we need to perform confusion analysis. maybe confusion matrices?
# to calculate confusion, we can make use of svm_predictions_output<C>.csv files
def calculate_confusion(filepath, classes):
    """
    argument:
        - filepath: path of a csv file composed of two columns, namely True_Label and Predicted_Label
    return:
        - confusion_dict: a dictionary in the form of `{<cls0_name-cls1_name>: <no_of_confused>}`
    """
    confusion_dict = {}
    for cls0 in classes:
        classes = np.delete(classes, np.where(classes == cls0))
        for cls1 in classes:
            confusion_dict[f"{cls0}-{cls1}"] = 0
    
    df = pd.read_csv(filepath)
    for row in df.itertuples(index=False):
        true = row[0]
        pred = row[1]

        if true != pred:
            true_cls = CLASSES[int(true)]
            pred_cls = CLASSES[int(pred)]
            confusion_dict[f"{true_cls}-{pred_cls}" if true < pred else f"{pred_cls}-{true_cls}"] += 1

    return confusion_dict

In [65]:
coonfusion_dict = calculate_confusion("/home/hdd/akademia/cmpe/s7/cmpe462/hw/1/cmpe-462/task_1/svm_predictions_output10.csv", CLASSES)
coonfusion_dict = dict(sorted(coonfusion_dict.items(), key=lambda pair: pair[1], reverse=True))

print("="*10, "CONFUSION", "="*10)
print(f"TEST SET SIZE: {len(Y_test)} samples")
print("-"*31)

for k, v in coonfusion_dict.items():
    print(f"{k:<{18}} | {v:>{2}}")

print("="*31)

========== CONFUSION ==========
TEST SET SIZE: 301 samples
-------------------------------
carrot-cucumber    | 15
cucumber-tomato    |  4
mandarin-tomato    |  4
banana-carrot      |  2
banana-cucumber    |  1
banana-mandarin    |  1
banana-tomato      |  0
carrot-mandarin    |  0
carrot-tomato      |  0
cucumber-mandarin  |  0


In [85]:
print("="*6, "DISTANCES & CONFUSION", "="*6)

for k, v in sv_dist_pw.items():
    print(f"{k:<{18}} | {v:.4f} | {coonfusion_dict[k]:>{2}}")

print("="*35)

====== DISTANCES & CONFUSION ======
carrot-cucumber    | 1.6727 | 15
mandarin-tomato    | 1.8552 |  4
carrot-mandarin    | 2.3193 |  0
banana-carrot      | 2.4131 |  2
cucumber-mandarin  | 2.7752 |  0
banana-cucumber    | 2.9387 |  1
carrot-tomato      | 3.2212 |  0
cucumber-tomato    | 3.4213 |  4
banana-mandarin    | 3.6384 |  1
banana-tomato      | 4.6404 |  0


_i am going to prepare a detailed explanation of why $\alpha_n = C$ identifies outliers_